# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
from utils import *

# Set your preferred name
USER = "Haoyang"
# Set the data and output directories
DATA_ROOT = '/media/techt/DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'COVID-19'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [ ]:
dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)

If no match is found, jump directly to GEO in Part 2.2

In [ ]:
trait_subdir = "TCGA_Breast_Cancer_(BRCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_BRCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.BRCA.sampleMap_HiSeqV2_PANCAN.gz')

In [ ]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)
clinical_data_cols[:10]

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

In [ ]:
candidate_age_cols = ['Age_at_Initial_Pathologic_Diagnosis_nature2012', 'age_at_initial_pathologic_diagnosis',
                      'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = ['Gender_nature2012', 'gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col=age_col, gender_col=gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [3]:
dataset = 'GEO'
trait_subdir = "COVID-19"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE207945', 'GSE243348']

Repeat the below steps for all the accession numbers

In [10]:
cohort = accession_num = "GSE243348"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/media/techt/DATA/GEO/COVID-19/GSE243348/GSE243348_family.soft.gz',
 '/media/techt/DATA/GEO/COVID-19/GSE243348/GSE243348_series_matrix.txt.gz')

### Initial filtering and clinical data preprocessing

In [11]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Longitudinal gene expression profiling of self-collected blood samples in COVID-19+ and healthy participants"
!Series_summary	"Longitudinal cohort: 773 host response genes were profiled in previously vaccinated (n=16) and unvaccinated (n=14) COVID-19+ participants along with 5 healthy uninfected controls across a 2-week observational window"
!Series_summary	"Single timepoint cohort: 773 host response genes were profiled in 6 healthy uninfected participants"
!Series_overall_design	"Longitudinal cohort: 30 COVID-19+ and 5 uninfected participants were asked perform self-collection and stabilization of capillary blood using a novel technology (homeRNA) every other day for two weeks (7 longtiudinal timepoints per participant). Temporal kinetics of 773 immune genes were profiled using the nCounter direct digital counting of native mRNA."
!Series_overall_design	"Single timepoint cohort: 6 healthy uninfected participants were asked perform self-collection and stabilization of ca

In [12]:
clinical_data.head()

,!Sample_geo_accession,GSM7783810,GSM7783811,GSM7783812,GSM7783813,GSM7783814,GSM7783815,GSM7783816,GSM7783817,GSM7783818,...,GSM7784037,GSM7784038,GSM7784039,GSM7784040,GSM7784041,GSM7784042,GSM7784043,GSM7784044,GSM7784045,GSM7784046
0,!Sample_characteristics_ch1,disease status: COVID-19+,disease status: COVID-19+,disease status: COVID-19+,disease status: COVID-19+,disease status: COVID-19+,disease status: COVID-19+,disease status: COVID-19+,disease status: COVID-19+,disease status: COVID-19+,...,disease status: Healthy uninfected,disease status: Healthy uninfected,disease status: Healthy uninfected,disease status: Healthy uninfected,disease status: Healthy uninfected,disease status: Healthy uninfected,disease status: Healthy uninfected,disease status: Healthy uninfected,disease status: Healthy uninfected,disease status: Healthy uninfected
1,!Sample_characteristics_ch1,participant id: CB0101,participant id: CB0101,participant id: CB0101,participant id: CB0101,participant id: CB0101,participant id: CB0101,participant id: CB0101,participant id: CB0102,participant id: CB0102,...,participant id: HC0105,participant id: HC0105,participant id: HC0105,participant id: HC0105,participant id: CA0101,participant id: CA0108,participant id: CA0112,participant id: CA0113,participant id: CA0119,participant id: CA0120
2,!Sample_characteristics_ch1,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female,...,Sex: female,Sex: female,Sex: female,Sex: female,Sex: male,Sex: female,Sex: female,Sex: female,Sex: male,Sex: male
3,!Sample_characteristics_ch1,age: 44,age: 44,age: 44,age: 44,age: 44,age: 44,age: 44,age: 29,age: 29,...,age: 40,age: 40,age: 40,age: 40,age: 36,age: 24,age: 28,age: 36,age: 27,age: 38
4,!Sample_characteristics_ch1,covid-19 vaccination history: unvaccinated,covid-19 vaccination history: unvaccinated,covid-19 vaccination history: unvaccinated,covid-19 vaccination history: unvaccinated,covid-19 vaccination history: unvaccinated,covid-19 vaccination history: unvaccinated,covid-19 vaccination history: unvaccinated,covid-19 vaccination history: unvaccinated,covid-19 vaccination history: unvaccinated,...,covid-19 vaccination history: vaccinated,covid-19 vaccination history: vaccinated,covid-19 vaccination history: vaccinated,covid-19 vaccination history: vaccinated,covid-19 vaccination history: vaccinated,covid-19 vaccination history: unvaccinated,covid-19 vaccination history: vaccinated,covid-19 vaccination history: unvaccinated,covid-19 vaccination history: unvaccinated,covid-19 vaccination history: vaccinated


In [13]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['disease status: COVID-19+', 'disease status: Healthy uninfected'],
 1: ['participant id: CB0101',
  'participant id: CB0102',
  'participant id: CB0104',
  'participant id: CB0106',
  'participant id: CB0107',
  'participant id: CB0111',
  'participant id: CB0112',
  'participant id: CB0113',
  'participant id: CB0115',
  'participant id: CB0116',
  'participant id: CB0117',
  'participant id: CB0118',
  'participant id: CB0119',
  'participant id: CB0120',
  'participant id: CB0121',
  'participant id: CB0122',
  'participant id: CB0123',
  'participant id: CB0124',
  'participant id: CB0125',
  'participant id: CB0128',
  'participant id: CB0129',
  'participant id: CB0130',
  'participant id: CB0131',
  'participant id: CB0132',
  'participant id: CB0133',
  'participant id: CB0134',
  'participant id: CB0135',
  'participant id: CB0136',
  'participant id: CB0138',
  'participant id: CB0139'],
 2: ['Sex: female', 'Sex: male'],
 3: ['age: 44',
  'age: 29',
  'age: 51',
  'age:

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [14]:

f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. Do remember to separate the value by colon ':' first, as the row header should not be included in the returned value. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'COVID-19\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'COVID-19\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer.

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [15]:
age_row = gender_row = None
convert_age = convert_gender = None

In [16]:
is_gene_availabe = True
trait_row = 0
age_row = 3
gender_row = 2

trait_type = 'binary'

In [17]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [18]:
# Verify and use the functions generated by GPT

def convert_trait(value):
    value = value.split(':')[1].strip()
    if value == 'COVID-19+':
        return 1
    elif value == 'Healthy uninfected':
        return 0
    else:
        return None

def convert_age(value):
    try:
        age = int(value.split(':')[1].strip())
        return age
    except ValueError:
        return None

def convert_gender(value):
    value = value.split(':')[1].strip()
    if value == 'female':
        return 0
    elif value == 'male':
        return 1
    else:
        return None


In [19]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM7783810,GSM7783811,GSM7783812,GSM7783813,GSM7783814,GSM7783815,GSM7783816,GSM7783817,GSM7783818,GSM7783819,...,GSM7784037,GSM7784038,GSM7784039,GSM7784040,GSM7784041,GSM7784042,GSM7784043,GSM7784044,GSM7784045,GSM7784046
COVID-19,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Age,44,44,44,44,44,44,44,29,29,29,...,40,40,40,40,36,24,28,36,27,38
Gender,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1


### Genetic data preprocessing and final filtering

In [20]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM7783810,GSM7783811,GSM7783812,GSM7783813,GSM7783814,GSM7783815,GSM7783816,GSM7783817,GSM7783818,GSM7783819,...,GSM7784037,GSM7784038,GSM7784039,GSM7784040,GSM7784041,GSM7784042,GSM7784043,GSM7784044,GSM7784045,GSM7784046
ID,,,,,,,,,,,,,,,,,,,,,
ACE,16.29,7.37,9.78,10.50,8.98,6.63,12.97,16.93,16.10,19.92,...,21.18,20.37,22.39,22.27,15.03,21.23,9.07,10.96,8.77,14.40
ACKR2,16.53,19.80,14.36,15.45,16.33,16.01,12.41,25.99,25.12,32.97,...,19.97,11.24,12.66,18.56,22.54,19.71,14.60,10.77,8.03,18.31
ACKR3,20.10,18.44,21.72,21.52,14.50,13.08,16.71,25.99,23.19,10.99,...,29.65,15.45,16.55,23.20,24.05,16.68,21.36,22.42,28.28,21.65
ACKR4,7.57,12.60,8.31,7.60,9.39,14.62,11.53,21.76,19.32,19.23,...,44.78,21.77,12.66,31.55,24.05,30.33,7.59,15.43,10.61,18.53
ACOX1,1054.81,1120.94,1648.33,1673.20,1338.32,1851.71,2153.50,1499.15,1475.12,1340.75,...,1045.69,1004.44,1040.78,1209.12,1199.25,1194.94,1088.64,1232.43,1094.51,975.54


In [21]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['ACE',
 'ACKR2',
 'ACKR3',
 'ACKR4',
 'ACOX1',
 'ACSL1',
 'ACSL3',
 'ACSL4',
 'ACVR1',
 'ADAR',
 'ADGRE5',
 'ADGRG3',
 'ADORA2A',
 'AGT',
 'AHR',
 'AIF1',
 'AIM2',
 'AKT1',
 'AKT2',
 'AKT3']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [22]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['ACE', 'ACKR2', 'ACKR3', 'ACKR4', 'ACOX1', 'ACSL1', 'ACSL3', 'ACSL4', 'ACVR1', 'ADAR', 'ADGRE5', 'ADGRG3', 'ADORA2A', 'AGT', 'AHR', 'AIF1', 'AIM2', 'AKT1', 'AKT2', 'AKT3']\n"


If not required, jump directly to the gene normalization step

In [23]:
requires_gene_mapping = False

In [24]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [25]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')

In [26]:
if requires_gene_mapping:
    identifier_key = 'ID'
    gene_symbol_key = 'Gene Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [27]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

1 input query terms found dup hits:	[('SIGLEC5', 2)]
20 input query terms found no hit:	['CARD17', 'CCL3/L1/L3', 'CCL4/L1/L2', 'CD45R0', 'CD45RA', 'CD45RB', 'DDX58', 'DEFB103A/B', 'FCGR1A/


In [28]:
genetic_data

,GSM7783810,GSM7783811,GSM7783812,GSM7783813,GSM7783814,GSM7783815,GSM7783816,GSM7783817,GSM7783818,GSM7783819,...,GSM7784037,GSM7784038,GSM7784039,GSM7784040,GSM7784041,GSM7784042,GSM7784043,GSM7784044,GSM7784045,GSM7784046
ACE,16.29,7.37,9.78,10.50,8.98,6.63,12.97,16.93,16.10,19.92,...,21.18,20.37,22.39,22.27,15.03,21.23,9.07,10.96,8.77,14.40
ACKR2,16.53,19.80,14.36,15.45,16.33,16.01,12.41,25.99,25.12,32.97,...,19.97,11.24,12.66,18.56,22.54,19.71,14.60,10.77,8.03,18.31
ACKR3,20.10,18.44,21.72,21.52,14.50,13.08,16.71,25.99,23.19,10.99,...,29.65,15.45,16.55,23.20,24.05,16.68,21.36,22.42,28.28,21.65
ACKR4,7.57,12.60,8.31,7.60,9.39,14.62,11.53,21.76,19.32,19.23,...,44.78,21.77,12.66,31.55,24.05,30.33,7.59,15.43,10.61,18.53
ACOX1,1054.81,1120.94,1648.33,1673.20,1338.32,1851.71,2153.50,1499.15,1475.12,1340.75,...,1045.69,1004.44,1040.78,1209.12,1199.25,1194.94,1088.64,1232.43,1094.51,975.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XBP1,1482.88,1728.78,1000.26,1187.84,1141.74,1075.05,875.83,1403.04,966.88,850.33,...,1051.14,967.22,1029.10,964.14,804.01,928.05,1186.09,1288.32,1105.08,1220.51
XCR1,10.11,12.47,9.10,5.01,12.16,21.93,13.69,29.02,36.72,19.92,...,24.21,26.69,46.73,46.40,36.07,36.39,10.70,17.73,14.89,26.12
YWHAQ,1804.95,1603.10,1458.75,1563.95,1511.85,1594.34,1377.09,1390.34,1655.49,1526.20,...,1308.93,1371.80,1631.76,1321.41,1523.86,1498.22,1111.90,1436.42,1406.97,1295.89
ZAP70,666.45,665.62,497.28,572.21,550.36,436.22,477.61,588.78,514.68,559.10,...,836.31,714.35,493.62,727.52,811.52,903.79,764.04,899.23,714.05,781.28


In [29]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [30]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 237 samples.


In [31]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'COVID-19', the least common label is '0.0' with 37 occurrences. This represents 15.61% of the dataset.
The distribution of the feature 'COVID-19' in this dataset is fine.

Quartiles for 'Age':
  25%: 30.0
  50% (Median): 36.0
  75%: 43.0
Min: 24.0
Max: 65.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 77 occurrences. This represents 32.49% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [32]:
if is_available:
    save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')
else:
    save_cohort_info(cohort, JSON_PATH, is_available)

A new JSON file was created at: ../../output1/Haoyang/COVID-19/cohort_info.json


In [33]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

### 3. Do regression & Cross Validation

In [34]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE243348,True,True,False,True,True,237,


In [35]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

'GSE243348'

In [36]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE243348,True,True,False,True,True,237,


In [37]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

,COVID-19,Age,Gender,ACE,ACKR2,ACKR3,ACKR4,ACOX1,ACSL1,ACSL3,...,VSIR,VWF,WAS,WIPI1,XAF1,XBP1,XCR1,YWHAQ,ZAP70,ZBP1
0,1.0,44.0,0.0,16.29,16.53,20.10,7.57,1054.81,3763.52,28.00,...,859.20,14.18,1346.80,33.66,819.23,1482.88,10.11,1804.95,666.45,573.54
1,1.0,44.0,0.0,7.37,19.80,18.44,12.60,1120.94,3348.47,29.11,...,697.07,17.83,1133.98,47.27,240.81,1728.78,12.47,1603.10,665.62,253.06
2,1.0,44.0,0.0,9.78,14.36,21.72,8.31,1648.33,5699.41,20.95,...,837.58,29.69,1510.44,48.17,255.67,1000.26,9.10,1458.75,497.28,278.85
3,1.0,44.0,0.0,10.50,15.45,21.52,7.60,1673.20,6284.57,28.35,...,1034.45,29.09,1587.53,39.70,216.95,1187.84,5.01,1563.95,572.21,260.41
4,1.0,44.0,0.0,8.98,16.33,14.50,9.39,1338.32,5751.24,19.11,...,908.03,15.34,1414.47,33.45,222.65,1141.74,12.16,1511.85,550.36,167.55


In [38]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [39]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [40]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [41]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 57.87% ± 7.17%


In [42]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 82.13% ± 6.25%


In [43]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 4. Discussion and report

In [44]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0729
Found 182 genes with non-zero coefficients associated with the trait 'COVID-19' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
Variable  Coefficient  Absolute Coefficient
   CCL18    -1.767921              1.767921
    LTBR    -1.551749              1.551749
    LEF1    -1.270181              1.270181
   SORT1    -1.269406              1.269406
   ACSL1    -1.085387              1.085387
  MAP3K3     1.057034              1.057034
HSP90AB1    -1.014894              1.014894
HLA-DQB1     0.957103              0.957103
     CD6    -0.943416              0.943416
   TRIM6    -0.892642              0.892642
    RELB    -0.880220              0.880220
    IL1A    -0.877533              0.877533
    SELL    -0.813514              0.813514
 IL12RB2     0.799333              0.799333
   IL6ST    -0.782577              0.782577
   P2RX7    -0.777539      

/home/techt/AI4Science_gold_standard/utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [45]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0729
Found 0 significant genes associated with the trait 'COVID-19' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, p_value, corrected_p_value]
Index: []


/home/techt/AI4Science_gold_standard/utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
